## Carga de datos

In [1]:
library(dplyr)
library(abind)
library(loadeR)
library(gridExtra)
library(loadeR.2nc)
library(visualizeR)
library(transformeR)
library(RColorBrewer)
library(easyVerification)
library(climate4R.indices)


Attaching package: ‘dplyr’




The following objects are masked from ‘package:stats’:

    filter, lag




The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




Loading required package: rJava



Loading required package: loadeR.java



...

climate4R.indices version 0.3.1 (2023-06-22) is loaded



Use 'indexShow()' for an overview of the available climate indices and circIndexShow() for the circulation indices.



NOTE: use package climate4R.climdex to calculate ETCCDI indices.




Attaching package: ‘climate4R.indices’




The following object is masked from ‘package:transformeR’:

    lambWT




In [2]:
# Color
color = colorRampPalette(rev(brewer.pal(n = 9, "RdYlBu")))

# Datos
pr_era5_1 = readRDS('../../../data/npd_s_w_complete/pr_npdw_era5_complete.rds')
pr_seas5_1 = readRDS('../../../data/npd_s_w_complete/pr_npdw_seas5_complete.rds')
pr_seas5_1_bc = readRDS('../../../data/npd_s_w_complete/pr_npdw_seas5_complete_bc.rds')

## Máscara para los datos

In [3]:
## Calculo el número de días que tmax > 25 grados (solo para la estructura del grid)
nd_obs = indexGrid(tx = pr_era5_1, index.code = "TXth", th = 25.3) %>% suppressMessages %>% suppressWarnings

## Máscara de tierra de ERA5 (es una variable más del propio reanális):
## Valores continuos entre 0 (no hay nada de tierra en ese gridbox) y 1 (todo el gridbox es tierra)
mask = loadGridData("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5/lsm/lsm_era5.nc", var = "lsm") %>% suppressMessages %>% suppressWarnings

## Binarizo la máscara: Considero que todos los gridboxes con un valor por encima (debajo) de 0.5 son de tierra (mar)
mask.bin = binaryGrid(mask, condition = "GE", threshold = 0.5, values = c(NA, 1))

## Hago el upscaling como hice con los datos de ERA5 a la resolución de 1º del modelo
mask_upscaled = interpGrid(mask.bin,
                           new.coordinates = getGrid(pr_era5_1),
                           method = "bilinear") %>% suppressMessages %>% suppressWarnings

## Apoyándome en la máscara binaria, me quedo únicamente con los datos en tierra y descarto el mar
mask.bin.spain = subsetGrid(mask_upscaled, lonLim = c(-10, 5), latLim = c(35, 44))
mask.bin.spain$Data = aperm(replicate(getShape(nd_obs)["time"], mask.bin.spain$Data, simplify = "array"), c(3, 1, 2))
attributes(mask.bin.spain$Data)$dimensions = c("time", "lat", "lon")

## Máscara para el moodelo
n.members = getShape(pr_seas5_1)["member"]
mask.data = mask.bin.spain$Data
mask.4d = array(NA, dim = c(n.members, dim(mask.data)))  # member x time x lat x lon
for (m in 1:n.members) {
    mask.4d[m,,,] = mask.data
}

mask.model = mask.bin.spain  # copia de la estructura
mask.model$Data = mask.4d
attributes(mask.model$Data)$dimensions = c("member", "time", "lat", "lon")

## Cálculo de índices

In [4]:
library(climate4R.agro)

climate4R.agro version 0.1.4 (2025-11-15) is loaded



Use 'agroindexShow()' for an overview of the available indices




Attaching package: ‘climate4R.agro’




The following objects are masked from ‘package:climate4R.indices’:

    agroindexFAO, agroindexFAO_tier1, avg, binSpell, dr, gsl, lds,
    nd_thre, nhw, nrd, ns, prcptot, prcptot_thre, sdii, yearStartEnd




In [5]:
calc_cdi_by_chunks = function(grid, start_year, end_year, chunk_size = 5) {
  
    # Crear secuencia de años
    year_seq = seq(start_year, end_year)
    
    # Crear tramos de chunk_size años
    chunks = split(year_seq, ceiling(seq_along(year_seq) / chunk_size))
    
    # Función para procesar cada tramo
    process_chunk = function(years) {
        
        y_start = min(years)
        y_end   = max(years)
        
        # Subset dinámico para el tramo
        sub = subsetGrid(grid, years = y_start:y_end)
        
        # Cálculo CEI
        agroindexGrid(
            index.code = "CDI",
            pr = sub,
            parallel = TRUE,
            index.arg.list = list(
                season_start  = "11-01",
                season_end    = "07-31",
                year_start    = y_start,
                year_end      = y_end,
                start_date    = as.Date(paste0(y_start, "-11-01")),
                bounds = data.frame(
                    var   = c("pr"),
                    lower = c(0),
                    inc_lower = c(FALSE)),
                combiner = "any",
                min_duration = 1)) %>% suppressMessages %>% suppressWarnings
    }
    
    # Procesar todos los tramos
    results = lapply(chunks, process_chunk)
    
    # Unir resultados en la dimensión temporal
    final = do.call(bindGrid, c(results, list(dimension = "time")))
    
    return(final)
}

In [6]:
index_obs = calc_cdi_by_chunks(
    grid = pr_era5_1,
    start_year = 1982,
    end_year = 2021,
    chunk_size = 1)

# Aplico máscara a los datos
index_obs_masked = gridArithmetics(index_obs, mask.bin.spain, operator = "*")
print(summary(as.vector(index_obs_masked$Data)))

# Guardo los indices en formato nc
grid2nc(data = index_obs_masked, NetCDFOutFile = "npd_w_era5_complete.nc")

saveRDS(index_obs_masked, "npd_w_era5_complete.rds")

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  206.0   241.0   250.0   248.9   258.0   272.0    3160 


[2025-12-15 15:41:25.851588] NetCDF file written in: npd_w_era5_complete.nc



In [7]:
index_raw = calc_cdi_by_chunks(
    grid = pr_seas5_1,
    start_year = 1982,
    end_year = 2021,
    chunk_size = 1)

# Aplico máscara a los datos
index_raw_masked = gridArithmetics(index_raw, mask.model, operator = "*")
print(summary(as.vector(index_raw_masked$Data)))

# Guardo los indices en formato nc
index_raw_masked$Members = as.character(seq_len(dim(index_raw_masked$Data)[1]))
grid2nc(data = index_raw_masked, NetCDFOutFile = "npd_w_seas5_complete_raw.nc")

saveRDS(index_raw_masked, "npd_w_seas5_complete_raw.rds")

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   75.0   160.0   185.0   186.7   216.0   263.0   79000 


[2025-12-15 16:07:31.470287] NetCDF file written in: npd_w_seas5_complete_raw.nc



In [8]:
index_bc = calc_cdi_by_chunks(
    grid = pr_seas5_1_bc,
    start_year = 1982,
    end_year = 2021,
    chunk_size = 1)

# Aplico máscara a los datos
index_bc_masked = gridArithmetics(index_bc, mask.model, operator = "*")
print(summary(as.vector(index_bc_masked$Data)))

# Guardo los indices en formato nc
index_bc_masked$Members = as.character(seq_len(dim(index_bc_masked$Data)[1]))
grid2nc(data = index_bc_masked, NetCDFOutFile = "npd_w_seas5_complete_bc.nc")

saveRDS(index_bc_masked, "npd_w_seas5_complete_bc.rds")

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   7.00   58.00   73.00   80.33   98.00  192.00   79000 


[2025-12-15 16:33:40.297731] NetCDF file written in: npd_w_seas5_complete_bc.nc

